In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').load('/FileStore/tables/sample_linear_regression_data.txt')

In [5]:
training.show()

+-------------------+--------------------+
 label| features|
+-------------------+--------------------+
 -9.490009878824548|(10,[0,1,2,3,4,5,...|
 0.2577820163584905|(10,[0,1,2,3,4,5,...|
 -4.438869807456516|(10,[0,1,2,3,4,5,...|
-19.782762789614537|(10,[0,1,2,3,4,5,...|
 -7.966593841555266|(10,[0,1,2,3,4,5,...|
 -7.896274316726144|(10,[0,1,2,3,4,5,...|
 -8.464803554195287|(10,[0,1,2,3,4,5,...|
 2.1214592666251364|(10,[0,1,2,3,4,5,...|
 1.0720117616524107|(10,[0,1,2,3,4,5,...|
-13.772441561702871|(10,[0,1,2,3,4,5,...|
 -5.082010756207233|(10,[0,1,2,3,4,5,...|
 7.887786536531237|(10,[0,1,2,3,4,5,...|
 14.323146365332388|(10,[0,1,2,3,4,5,...|
-20.057482615789212|(10,[0,1,2,3,4,5,...|
-0.8995693247765151|(10,[0,1,2,3,4,5,...|
 -19.16829262296376|(10,[0,1,2,3,4,5,...|
 5.601801561245534|(10,[0,1,2,3,4,5,...|
-3.2256352187273354|(10,[0,1,2,3,4,5,...|
 1.5299675726687754|(10,[0,1,2,3,4,5,...|
 -0.250102447941961|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 20 rows

In [6]:
lr = LinearRegression(featuresCol='features',labelCol='label',
                     predictionCol='prediction')

In [7]:
lrModel = lr.fit(training)

In [8]:
lrModel.coefficients

Out[ 8 ]: DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [9]:
lrModel.intercept

Out[ 10 ]: 0.14228558260358093

In [10]:
training_summary = lrModel.summary

In [11]:
#r-squared value
training_summary.r2

Out[ 16 ]: 0.027839179518600154

In [12]:
training_summary.rootMeanSquaredError

Out[ 17 ]: 10.16309157133015

In [13]:
training_summary.meanSquaredError

Out[ 20 ]: 103.28843028724194

In [14]:
all_data = spark.read.format('libsvm').load('/FileStore/tables/sample_linear_regression_data.txt')

In [15]:
# tuple unpacking to split training and test data from the dataframe
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [16]:
train_data

Out[ 30 ]: DataFrame[label: double, features: vector]

In [17]:
test_data

Out[ 31 ]: DataFrame[label: double, features: vector]

In [18]:
train_data.describe().show()

+-------+-------------------+
summary| label|
+-------+-------------------+
 count| 378|
 mean| 0.5689089532590789|
 stddev| 10.388503560857895|
 min|-28.571478869743427|
 max| 27.78383192005107|
+-------+-------------------+

In [19]:
test_data.describe().show()

+-------+-------------------+
summary| label|
+-------+-------------------+
 count| 123|
 mean|-0.7020023122946534|
 stddev| 10.078889731014607|
 min|-26.736207182601724|
 max| 21.57719950299147|
+-------+-------------------+

In [20]:
# fit training data to the lr created earlier
correct_model = lr.fit(train_data)

In [21]:
# pass the test_data DF into `evaluate` method of the train_data fitted model
test_results = correct_model.evaluate(test_data)

In [22]:
test_results.rootMeanSquaredError

Out[ 42 ]: 10.228122804453006

In [23]:
# usually will deploy a model you're satisfied with with data that is unlabeled, so making that here
unlabeled_data = test_data.select('features')

In [24]:
unlabeled_data.show(5)

+--------------------+
 features|
+--------------------+
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 5 rows

In [25]:
predictions = correct_model.transform(unlabeled_data)

In [26]:
predictions.show(5)

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
(10,[0,1,2,3,4,5,...|-3.434346526850886|
(10,[0,1,2,3,4,5,...|4.3958503401568825|
(10,[0,1,2,3,4,5,...|-3.144517580432058|
(10,[0,1,2,3,4,5,...|1.5675355041739483|
(10,[0,1,2,3,4,5,...|1.0443001128181928|
+--------------------+------------------+
only showing top 5 rows